#Unscented Kalman Filter (additive noise) Example

###Introduction

This notebook is designed to demonstrate how to use the StateSpace.jl package to execute the Unscented Kalman filter with additive noise for a non-linear State Space model. The example that has been used here closely follows the example given in a paper by Rambabu Kandepu, Bjarne Foss and Lars Imsland ["Applying the unscented Kalman filter for nonlinear state estimation".](http://folk.ntnu.no/bjarnean/pubs/journals/journal-47.pdf) The example is the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) (Section 4.1 on page 6 of the paper). You can read the paper for full details.   

For those of you that do not need/want the explanation of the model and the code, you can skip right to the end of this notebook where the entire section of code required to run this example is given.

### The Problem

The problem that we will consider here is that of predicting the position $\mathbf{x} = (x_1, x_2)$ of a Van der Pol oscillator given some noisy measurements of its position.

####The Process Model
The dynamic equations describing the rate of change of position of the oscillator are:

$$
\begin{align}
\frac{\mathrm{d}x_1}{\mathrm{d}t} &= -x_2, \\
\frac{\mathrm{d}x_2}{\mathrm{d}t} &= -\mu \left(1 - x_1^2 \right)x_2 + x_1, \\
\end{align}
$$

where $\mu$ is a scalar parameter that defines the strength of the damping.   
These equations can be written in a discrete manner required for solution with the unscented Kalman filter as follows: 

$$
\begin{align}
x_1^n &= x_1^{n-1} - \Delta t \times x_2^{n-1}, \\
x_2^n &= x_2^{n-1} - \Delta t \times \mu \left(1 - (x_1^{n-1})^2 \right)x_2^{n-1} + x_1^{n-1}, \\
\end{align}
$$

where $\mathbf{x}^n = \left( x_1^n, x_2^n \right)$ represents the position of the oscillator at time step $n$ and $\Delta t$ is the time step between consecutive measurements.

#### The Observation Model
We assume that we measure the position of the oscillator directly and hence the observation function is simply
$$
y^n(\mathbf{x}^n) = \left( x_1^n, x_2^n \right) 
$$

###Setting up the problem
First we'll import the required modules

In [1]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

#####Define Kalman Filter Parameters
We now need to define the process and observation model according to the model described above.

In [12]:
Δt = 0.1 # Set the time step

#Set the process function
function processFunction(x::Vector, Δt::Float64=0.1, μ::Float64=0.3)
    x1 = zeros(x)
    x1[1] = x[1] + Δt * -x[2]
    x1[2] = x[2] + Δt * (-μ * (1 - x[1]^2) * x[2] + x[1])
    x1
end

#Set the observation function.
observationFunction(x::Vector) = x

#Set process noise covariance matrix
processCovariance = 1e-1*[0.1 0.0; 0.0 1e-3]

#Set the observation noise covariance
observationCovariance = 3e-2*eye(2)

#Create additive noise UKF model
ukfStateModel = AdditiveNonLinUKFSSM(processFunction, processCovariance, observationFunction, observationCovariance)

StateSpace.AdditiveNonLinUKFSSM{Float64}(processFunction,2x2 Array{Float64,2}:
 0.01  0.0   
 0.0   0.0001,observationFunction,2x2 Array{Float64,2}:
 0.03  0.0 
 0.0   0.03)

#####Generate noisy observations
Here we'll generate the noisy observations that will be passed to the Unscented Kalman Filter.

First we define the true initial position of the system

In [13]:
trueInitialState = [1.4, 0.0] #Set the true initial state of the oscillator

2-element Array{Float64,1}:
 1.4
 0.0

Now we can define the number of observations and generate the true and noisy observations

In [14]:
#Set the number of observations and generate the true values and the noisy
#observations.
numObs = 200
trueState = zeros(2,numObs)
noisyObs = zeros(2,numObs)
trueState[:,1] = trueInitialState
noisyObs[:,1] = trueInitialState + sqrt(observationCovariance)*randn(2)
for i in 2:numObs
    trueState[:,i] = processFunction(trueState[:,i-1])
    noisyObs[:,i] = trueState[:,i] + sqrt(observationCovariance)*randn(2)
end

####Guess of initial state
Now we define the guess of the initial state of the system

In [15]:
initial_guess = MvNormal([0.0,2.0], 1.0*eye(2))

FullNormal(
dim: 2
μ: [0.0,2.0]
Σ: 2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
)


#####Perform Unscented Kalman Filter (additive noise) Algorithm
Now we have all of the parameters:
1. noisy observations
2. process (transition) and observation (emission) model paramaters
3. initial guess of state   

We can use the Unscented Kalman Filter to predict the true underlying state (position of the oscillator).

In [16]:
filtered_state = filter(ukfStateModel, noisyObs, initial_guess)

SmoothedState{Float64}


201 estimates of 2-D process from 2-D observations
Log-likelihood: 701.8763985625116


NOTE: The log likelihood hasn't been implemented for the Unscented Kalman Filter and so 0.0 is just the default value set. It doesn't mean anything yet. We will sort this out soon. 

###Plot the results
Now we will plot the results of the $x_1$ position of the oscillator.

First we extract the mean $x_1$ positions and their variances from the filtered estimates. 

In [17]:
x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = filtered_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end

Next we will create a dataframe. This is simply so the syntax is simple for plotting the ribbon digram which will represent the state along with the 95% confidence interval

In [18]:
df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Filtered values"
    )

,x,y,ymin,ymax,f
1,0.1,0.0,-2.0,2.0,Filtered values
2,0.2,1.4082183222429558,1.0125689430929183,1.8038677013929934,Filtered values
3,0.30000000000000004,1.3270343378454568,0.9983130422356324,1.6557556334552812,Filtered values
4,0.4,1.3669012029391918,1.055790931877306,1.6780114740010776,Filtered values
5,0.5,1.2077546471196439,0.9022328133118163,1.5132764809274715,Filtered values
6,0.6,1.2217532103709878,0.9181560197247857,1.52535040101719,Filtered values
7,0.7000000000000001,1.031687856399231,0.7288253108774647,1.3345504019209975,Filtered values
8,0.8,1.0947832148967058,0.7921859500532789,1.3973804797401328,Filtered values
9,0.9,0.9258291881280865,0.6234388909948582,1.2282194852613149,Filtered values
10,1.0,1.0220403440150887,0.7197155623609277,1.3243651256692497,Filtered values


Next we create 3 distinguishable colors for the plot

In [19]:
n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

Finally we display the plot

In [25]:
oscillatorx1_state_plot = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Filter (Additive) Example")
    )
display(oscillatorx1_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,49.54 L 91.52 47.72 91.16 47.11 90.8 46.55 90.44 45.43 90.07 44.05 89.71 43.62 89.35 42.69 88.99 42.78 88.62 42.64 88.26 42.46 87.9 42.01 87.54 42.33 87.17 43.2 86.81 43.64 86.45 43.44 86.09 43.79 85.72 44.38 85.36 45.01 85 45.51 84.64 46.32 84.27 46.53 83.91 47.5 83.55 47.96 83.19 48.75 82.83 49.53 82.46 50.68 82.1 51.65 81.74 51.59 81.38 52.14 81.01 53.06 80.65 54.3 80.29 55 79.93 55.17 79.56 56.67 79.2 57.22 78.84 57.78 78.48 57.89 78.11 57.66 77.75 58 77.39 58.51 77.03 58.37 76.66 58.47 76.3 58.72 75.94 59.21 75.58 59.73 75.21 59.94 74.85 59.89 74.49 59.53 74.13 58.27 73.76 56.77 73.4 55.39 73.04 54.29 72.68 53.19 72.31 52.01 71.95 51.05 71.59 49.9 71.23 49.1 70.86 48.55 70.5 47.91 70.14 47.56 69.78 46.76 69.41 45.99 69.05 45.16 68.69 43.52 68.33 42.29 67.97 41.92 67.6 41.53 67.24 41.1 66.88 40.24 66.52 39.68 66.15 39.35 65.79 38.6 65.43 38.06 65.07 37.54 64.7 37.9 64.34 38.3 63.98 39.26 63.62 39.57 63.25 39.95 62.89 40.75 62.53 41.53 62.17 42.02 61.8 43.42 61.44 44.67 61.08 45.4 60.72 46.34 60.35 47.31 59.99 48.4 59.63 49.62 59.27 51.41 58.9 52.72 58.54 53.95 58.18 55.32 57.82 57.14 57.45 58.79 57.09 60.91 56.73 62.43 56.37 63.42 56 64.04 55.64 64.92 55.28 65.48 54.92 65.73 54.55 66.11 54.19 65.93 53.83 65.45 53.47 65.48 53.11 65.52 52.74 65.06 52.38 64.55 52.02 63.84 51.66 62.28 51.29 61.25 50.93 59.98 50.57 59.04 50.21 58.33 49.84 56.53 49.48 55.17 49.12 53.91 48.76 52.65 48.39 50.54 48.03 48.57 47.67 46.79 47.31 45.22 46.94 42.86 46.58 41.06 46.22 39.39 45.86 38.56 45.49 37.28 45.13 36.06 44.77 34.6 44.41 33.53 44.04 32.56 43.68 31.97 43.32 31.84 42.96 32.64 42.59 32.85 42.23 32.86 41.87 33.56 41.51 34.05 41.14 34.38 40.78 34.93 40.42 36.07 40.06 37.37 39.69 38.88 39.33 40.87 38.97 42.52 38.61 44.17 38.25 45.79 37.88 47.06 37.52 48.36 37.16 49.72 36.8 51.45 36.43 53.81 36.07 56.21 35.71 58.43 35.35 60.66 34.98 62.88 34.62 64.46 34.26 65.73 33.9 66.89 33.53 67.56 33.17 68.04 32.81 68.84 32.45 69.83 32.08 70.16 31.72 70.51 31.36 70.39 31 70.55 30.63 70.28 30.27 69.62 29.91 68.97 29.55 67.72 29.18 66.21 28.82 64.77 28.46 63.03 28.1 61.58 27.73 59.87 27.37 58.59 27.01 56.77 26.65 54.74 26.28 53 25.92 51.45 25.56 49.02 25.2 46.64 24.83 45.29 24.47 42.67 24.11 40.06 23.75 37.75 23.39 35.63 23.02 33.84 22.66 33.04 22.3 31.9 21.94 31.62 21.57 30.81 21.21 30.64 20.85 29.98 20.49 29.88 20.12 29.95 19.76 31.31 19.76 21.36 20.12 21.95 20.49 22.51 20.85 22.82 21.21 23.57 21.57 23.77 21.94 24.6 22.3 24.9 22.66 26.05 23.02 26.87 23.39 28.66 23.75 30.76 24.11 33.02 24.47 35.58 24.83 38.13 25.2 39.45 25.56 41.78 25.92 44.17 26.28 45.69 26.65 47.41 27.01 49.43 27.37 51.23 27.73 52.5 28.1 54.

###Unscented Rauch-Tung-Striebel Smoother

The Unscented RTS smoother has been implemented to perform smoothing on the filtered estimates to get a better estimate of the true underlying state (as they say, hindsight is 20/20). This can be performed in an identical manner to the Linear case like so:

In [21]:
smoothed_state = smooth(ukfStateModel, filtered_state)

SmoothedState{Float64}


200 estimates of 2-D process from 2-D observations
Log-likelihood: 0.0


Now we can the results in the same way as above to visualize the smoothed result.

In [23]:
x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = smoothed_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Smoothed values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))
oscillatorx1_state_plot_smooted = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Smoother (Additive) Example")
    )
display(oscillatorx1_state_plot_smooted)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,49.54 L 91.52 47.72 91.16 47.11 90.8 46.55 90.44 45.43 90.07 44.05 89.71 43.62 89.35 42.69 88.99 42.78 88.62 42.64 88.26 42.46 87.9 42.01 87.54 42.33 87.17 43.2 86.81 43.64 86.45 43.44 86.09 43.79 85.72 44.38 85.36 45.01 85 45.51 84.64 46.32 84.27 46.53 83.91 47.5 83.55 47.96 83.19 48.75 82.83 49.53 82.46 50.68 82.1 51.65 81.74 51.59 81.38 52.14 81.01 53.06 80.65 54.3 80.29 55 79.93 55.17 79.56 56.67 79.2 57.22 78.84 57.78 78.48 57.89 78.11 57.66 77.75 58 77.39 58.51 77.03 58.37 76.66 58.47 76.3 58.72 75.94 59.21 75.58 59.73 75.21 59.94 74.85 59.89 74.49 59.53 74.13 58.27 73.76 56.77 73.4 55.39 73.04 54.29 72.68 53.19 72.31 52.01 71.95 51.05 71.59 49.9 71.23 49.1 70.86 48.55 70.5 47.91 70.14 47.56 69.78 46.76 69.41 45.99 69.05 45.16 68.69 43.52 68.33 42.29 67.97 41.92 67.6 41.53 67.24 41.1 66.88 40.24 66.52 39.68 66.15 39.35 65.79 38.6 65.43 38.06 65.07 37.54 64.7 37.9 64.34 38.3 63.98 39.26 63.62 39.57 63.25 39.95 62.89 40.75 62.53 41.53 62.17 42.02 61.8 43.42 61.44 44.67 61.08 45.4 60.72 46.34 60.35 47.31 59.99 48.4 59.63 49.62 59.27 51.41 58.9 52.72 58.54 53.95 58.18 55.32 57.82 57.14 57.45 58.79 57.09 60.91 56.73 62.43 56.37 63.42 56 64.04 55.64 64.92 55.28 65.48 54.92 65.73 54.55 66.11 54.19 65.93 53.83 65.45 53.47 65.48 53.11 65.52 52.74 65.06 52.38 64.55 52.02 63.84 51.66 62.28 51.29 61.25 50.93 59.98 50.57 59.04 50.21 58.33 49.84 56.53 49.48 55.17 49.12 53.91 48.76 52.65 48.39 50.54 48.03 48.57 47.67 46.79 47.31 45.22 46.94 42.86 46.58 41.06 46.22 39.39 45.86 38.56 45.49 37.28 45.13 36.06 44.77 34.6 44.41 33.53 44.04 32.56 43.68 31.97 43.32 31.84 42.96 32.64 42.59 32.85 42.23 32.86 41.87 33.56 41.51 34.05 41.14 34.38 40.78 34.93 40.42 36.07 40.06 37.37 39.69 38.88 39.33 40.87 38.97 42.52 38.61 44.17 38.25 45.79 37.88 47.06 37.52 48.36 37.16 49.72 36.8 51.45 36.43 53.81 36.07 56.21 35.71 58.43 35.35 60.66 34.98 62.88 34.62 64.46 34.26 65.73 33.9 66.89 33.53 67.56 33.17 68.04 32.81 68.84 32.45 69.83 32.08 70.16 31.72 70.51 31.36 70.39 31 70.55 30.63 70.28 30.27 69.62 29.91 68.97 29.55 67.72 29.18 66.21 28.82 64.77 28.46 63.03 28.1 61.58 27.73 59.87 27.37 58.59 27.01 56.77 26.65 54.74 26.28 53 25.92 51.45 25.56 49.02 25.2 46.64 24.83 45.29 24.47 42.67 24.11 40.06 23.75 37.75 23.39 35.63 23.02 33.84 22.66 33.04 22.3 31.9 21.94 31.62 21.57 30.81 21.21 30.64 20.85 29.98 20.49 29.88 20.12 29.95 19.76 31.31 19.76 21.36 20.12 21.95 20.49 22.51 20.85 22.82 21.21 23.57 21.57 23.77 21.94 24.6 22.3 24.9 22.66 26.05 23.02 26.87 23.39 28.66 23.75 30.76 24.11 33.02 24.47 35.58 24.83 38.13 25.2 39.45 25.56 41.78 25.92 44.17 26.28 45.69 26.65 47.41 27.01 49.43 27.37 51.23 27.73 52.5 28.1 54.

For those that just want the code, without the explanation, you have come to the right place. Here it is:

In [26]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

Δt = 0.1 # Set the time step

#Set the process function
function processFunction(x::Vector, Δt::Float64=0.1, μ::Float64=0.3)
    x1 = zeros(x)
    x1[1] = x[1] + Δt * -x[2]
    x1[2] = x[2] + Δt * (-μ * (1 - x[1]^2) * x[2] + x[1])
    x1
end

#Set the observation function.
observationFunction(x::Vector) = x

#Set process noise covariance matrix
processCovariance = 1e-1*[0.1 0.0; 0.0 1e-3]

#Set the observation noise covariance
observationCovariance = 3e-2*eye(2)

#Create additive noise UKF model
ukfStateModel = AdditiveNonLinUKFSSM(processFunction, processCovariance, observationFunction, observationCovariance)

trueInitialState = [1.4, 0.0] #Set the true initial state of the oscillator

#Set the number of observations and generate the true values and the noisy
#observations.
numObs = 200
trueState = zeros(2,numObs)
noisyObs = zeros(2,numObs)
trueState[:,1] = trueInitialState
noisyObs[:,1] = trueInitialState + sqrt(observationCovariance)*randn(2)
for i in 2:numObs
    trueState[:,i] = processFunction(trueState[:,i-1])
    noisyObs[:,i] = trueState[:,i] + sqrt(observationCovariance)*randn(2)
end

initial_guess = MvNormal([0.0,2.0], 1.0*eye(2))

filtered_state = filter(ukfStateModel, noisyObs, initial_guess)

x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = filtered_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end

df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Filtered values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

oscillatorx1_state_plot = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Filter (Additive) Example")
    )
display(oscillatorx1_state_plot)

#Perform Smoothing of the filtered estimates
smoothed_state = smooth(ukfStateModel, filtered_state)

#Plot the results
x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = smoothed_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Smoothed values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))
oscillatorx1_state_plot_smooted = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Smoother (Additive) Example")
    )
display(oscillatorx1_state_plot_smooted)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,48.21 L 91.52 45.99 91.16 44.92 90.8 46.38 90.44 46.39 90.07 44.47 89.71 47.62 89.35 46.28 88.99 46.12 88.62 44.52 88.26 45.83 87.9 43.68 87.54 44.92 87.17 44.49 86.81 43.58 86.45 42.83 86.09 44.04 85.72 44.87 85.36 46.16 85 45.86 84.64 45.1 84.27 47.93 83.91 48.38 83.55 48.63 83.19 48.87 82.83 48.29 82.46 49.61 82.1 50.48 81.74 51.28 81.38 54.11 81.01 54.38 80.65 54.76 80.29 54.9 79.93 55.42 79.56 59.97 79.2 58.08 78.84 60.49 78.48 60.66 78.11 59.06 77.75 59.8 77.39 61.2 77.03 60.34 76.66 59.76 76.3 59.82 75.94 60.56 75.58 62.48 75.21 61.49 74.85 59.32 74.49 57.34 74.13 58.51 73.76 57.06 73.4 56.07 73.04 54.77 72.68 53.38 72.31 54.81 71.95 53.99 71.59 52.21 71.23 52.87 70.86 54.44 70.5 50.81 70.14 49.22 69.78 50.93 69.41 49.91 69.05 47.46 68.69 46.15 68.33 46.25 67.97 42.96 67.6 43.29 67.24 42.08 66.88 40.12 66.52 37.32 66.15 37.96 65.79 37.93 65.43 39.66 65.07 41.19 64.7 39.86 64.34 39.91 63.98 39.4 63.62 41.29 63.25 43.05 62.89 42.45 62.53 43.84 62.17 44.75 61.8 45.6 61.44 46.64 61.08 47.85 60.72 48.99 60.35 49.23 59.99 51.11 59.63 50.03 59.27 51.26 58.9 55.12 58.54 58.21 58.18 59.08 57.82 57.49 57.45 59.36 57.09 61.36 56.73 63.84 56.37 63.12 56 62.88 55.64 64.67 55.28 65.05 54.92 65.16 54.55 66.81 54.19 66.11 53.83 67.42 53.47 66.01 53.11 67.01 52.74 66.05 52.38 65.45 52.02 64.76 51.66 62.62 51.29 61.63 50.93 59.57 50.57 58.88 50.21 56.12 49.84 56.1 49.48 57.32 49.12 56.84 48.76 53.35 48.39 52.13 48.03 50.23 47.67 48.61 47.31 46.5 46.94 43.69 46.58 43.42 46.22 42.16 45.86 41.27 45.49 37.36 45.13 35.82 44.77 35.01 44.41 35.24 44.04 33.96 43.68 33.55 43.32 31.93 42.96 32.34 42.59 33.22 42.23 33.52 41.87 32.03 41.51 33.13 41.14 34.84 40.78 34.52 40.42 34.13 40.06 36.21 39.69 38.2 39.33 40.41 38.97 43.08 38.61 45.28 38.25 44.4 37.88 48.97 37.52 51.02 37.16 50.52 36.8 55.23 36.43 55.76 36.07 57.48 35.71 60.64 35.35 61.91 34.98 63.55 34.62 63.41 34.26 64.69 33.9 64.46 33.53 68.98 33.17 69.02 32.81 69.54 32.45 70.53 32.08 74.06 31.72 72.91 31.36 71.29 31 69.61 30.63 69.31 30.27 71.34 29.91 71.62 29.55 70.29 29.18 69.12 28.82 68.45 28.46 64.87 28.1 64.98 27.73 64.12 27.37 61.24 27.01 58.57 26.65 55.19 26.28 52.57 25.92 50.95 25.56 49.58 25.2 48.37 24.83 46.24 24.47 43.37 24.11 41.41 23.75 39.02 23.39 36.1 23.02 35.57 22.66 36.03 22.3 32.84 21.94 30.02 21.57 29.16 21.21 31.37 20.85 30.14 20.49 31.4 20.12 33.13 19.76 78.71 19.76 14.61 20.12 20.45 20.49 20.86 20.85 20.16 21.21 21.57 21.57 19.42 21.94 20.31 22.3 23.14 22.66 26.34 23.02 25.88 23.39 26.42 23.75 29.34 24.11 31.73 24.47 33.69 24.83 36.55 25.2 38.68 25.56 39.88 25.92 41.25 26.28 42.86 26.65 45.48 27.01 48.86 27.37 51.52 27.73 5

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,49.27 L 91.52 47.34 91.16 45.62 90.8 44.88 90.44 44.43 90.07 43.72 89.71 44.2 89.35 44.25 88.99 44.32 88.62 43.97 88.26 44.12 87.9 43.68 87.54 43.71 87.17 43.63 86.81 43.34 86.45 43.08 86.09 43.22 85.72 43.57 85.36 44.3 85 44.71 84.64 44.94 84.27 45.81 83.91 46.57 83.55 47.23 83.19 47.7 82.83 47.92 82.46 48.41 82.1 49.05 81.74 49.89 81.38 51.43 81.01 52.68 80.65 53.66 80.29 54.49 79.93 55.17 79.56 56.78 79.2 57.35 78.84 58.33 78.48 59.04 78.11 59.01 77.75 59.15 77.39 59.63 77.03 59.73 76.66 59.53 76.3 59.31 75.94 59.27 75.58 59.73 75.21 59.71 74.85 58.96 74.49 57.78 74.13 57.3 73.76 56.46 73.4 55.62 73.04 54.61 72.68 53.36 72.31 52.75 71.95 52.08 71.59 51.09 71.23 50.56 70.86 50.61 70.5 49.66 70.14 48.51 69.78 48.06 69.41 47.57 69.05 46.5 68.69 45.33 68.33 44.58 67.97 43.03 67.6 42.08 67.24 41.06 66.88 39.8 66.52 38.17 66.15 37.23 65.79 36.65 65.43 36.72 65.07 37.33 64.7 37.59 64.34 37.98 63.98 38.19 63.62 39.01 63.25 40.22 62.89 40.98 62.53 42.1 62.17 43.17 61.8 44.24 61.44 45.33 61.08 46.61 60.72 47.9 60.35 48.9 59.99 50.2 59.63 50.77 59.27 51.48 58.9 52.97 58.54 54.94 58.18 56.66 57.82 57.36 57.45 58.51 57.09 59.76 56.73 61.28 56.37 62.1 56 62.54 55.64 63.2 55.28 63.78 54.92 64.01 54.55 64.57 54.19 64.57 53.83 64.82 53.47 64.59 53.11 64.74 52.74 64.41 52.38 63.89 52.02 63.19 51.66 62.07 51.29 60.82 50.93 59.38 50.57 58.13 50.21 56.41 49.84 55.06 49.48 54.49 49.12 53.9 48.76 52.42 48.39 51 48.03 49.62 47.67 48.18 47.31 46.53 46.94 44.62 46.58 43.1 46.22 41.6 45.86 40.18 45.49 38.13 45.13 36.21 44.77 34.76 44.41 33.94 44.04 33.08 43.68 32.51 43.32 31.78 42.96 31.45 42.59 31.67 42.23 32.06 41.87 32.01 41.51 32.38 41.14 33.23 40.78 33.84 40.42 34.29 40.06 35.31 39.69 36.72 39.33 38.47 38.97 40.56 38.61 42.75 38.25 44.14 37.88 46.46 37.52 48.93 37.16 50.45 36.8 52.88 36.43 54.78 36.07 56.56 35.71 58.67 35.35 60.62 34.98 62.5 34.62 63.67 34.26 64.74 33.9 65.28 33.53 66.7 33.17 67.76 32.81 68.71 32.45 69.24 32.08 70.54 31.72 71.09 31.36 70.9 31 70.22 30.63 69.41 30.27 69.28 29.91 69.2 29.55 68.56 29.18 67.61 28.82 66.7 28.46 64.89 28.1 63.57 27.73 62.29 27.37 60.43 27.01 58.25 26.65 55.69 26.28 53.05 25.92 50.53 25.56 48.24 25.2 46.29 24.83 44.3 24.47 42.21 24.11 40.22 23.75 38.01 23.39 35.49 23.02 33.62 22.66 32.67 22.3 31.36 21.94 29.7 21.57 28.48 21.21 28.44 20.85 28.25 20.49 28.62 20.12 29.39 19.76 30.93 19.76 20.88 20.12 21.3 20.49 21.16 20.85 21.03 21.21 21.33 21.57 21.44 21.94 22.69 22.3 24.37 22.66 25.68 23.02 26.63 23.39 28.51 23.75 31 24.11 33.17 24.47 35.12 24.83 37.16 25.2 39.11 25.56 41.02 25.92 43.27 26.28 45.76 26.65 48.37 27.01 50.9 27.37 53.06 27.73 54.91 28.1 56.19 28